# Evaluation with Data
In this notebook, we introduce built-in evaluators and guide you through creating your own custom evaluators. We'll cover both code-based and prompt-based custom evaluators. Finally, we'll demonstrate how to use the `evaluate` API to assess data using these evaluators.


## 1. Built-in Evaluators

The table below lists all the built-in evaluators we support. In the following sections, we will select a few of these evaluators to demonstrate how to use them.


| Category       | Namespace                                        | Evaluator Class           |
|----------------|--------------------------------------------------|---------------------------|
| Quality        | promptflow.evals.evaluators                      | GroundednessEvaluator     |
|                |                                                  | RelevanceEvaluator        |
|                |                                                  | CoherenceEvaluator        |
|                |                                                  | FluencyEvaluator          |
|                |                                                  | SimilarityEvaluator       |
|                |                                                  | F1ScoreEvaluator          |
| Content Safety | promptflow.evals.evaluators.content_safety       | ViolenceEvaluator         |
|                |                                                  | SexualEvaluator           |
|                |                                                  | SelfHarmEvaluator         |
|                |                                                  | HateUnfairnessEvaluator   |
| Composite      | promptflow.evals.evaluators                      | QAEvaluator               |
|                |                                                  | ChatEvaluator             |
|                |                                                  | ContentSafetyEvaluator    |


### 1.1 Quality Evaluator

In [ ]:
import os
from promptflow.core import AzureOpenAIModelConfiguration

# Initialize Azure OpenAI Connection
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"),
    azure_deployment="gpt-35-turbo",
    api_version="2024-02-15-preview"
)

In [ ]:
from promptflow.evals.evaluators import RelevanceEvaluator

# Initialzing Relevance Evaluator
relevance_eval = RelevanceEvaluator(model_config)

In [ ]:
# Running Relevance Evaluator on single input row
relevance_score = relevance_eval(
    answer="The Alpine Explorer Tent is the most waterproof.",
    context="From the our product list,"
    " the alpine explorer tent is the most waterproof."
    " The Adventure Dining Table has higher weight.",
    question="Which tent is the most waterproof?",
)

In [ ]:
print(relevance_score)

### 1.2 Content Safety Evaluator

Unlike quality evaluators, which are prompt-based, content safety evaluators are supported by the RAI service. To use these evaluators, you need to provide the subscription ID, resource group, and project name of your Azure AI project.

In [ ]:
# AI Project Scope
project_scope = {
    "subscription_id": "e0fd569c-e34a-4249-8c24-e8d723c7f054",
    "resource_group_name": "resource-group",
    "project_name": "project-name",
}

In [ ]:
from promptflow.evals.evaluators.content_safety import ViolenceEvaluator

# Initialzing Violence Evaluator
violence_eval = ViolenceEvaluator(project_scope)

In [ ]:
# Running Violence Evaluator on single input row
violence_score = violence_eval(question="What is the capital of France?", answer="Paris.")

In [ ]:
print(violence_score)

### 1.3 Composite Evaluator

#### 1.3.1 QA Evaluator
The QAEvaluator is a composite evaluator built on top of individual quality evaluators. It provides a combined result from these underlying evaluators.

In [ ]:
from promptflow.evals.evaluators import QAEvaluator

# Initialzing QA Evaluator
qa_eval = QAEvaluator(model_config=model_config)

In [ ]:
# Running QA Evaluator on single input row
score = qa_eval(
    question="Tokyo is the capital of which country?",
    answer="Japan",
    context="Tokyo is the capital of Japan.",
    ground_truth="Japan",
)

In [ ]:
print(score)

#### 1.3.2 Chat Evaluator
The Chat Evaluator is another composite evaluator that utilizes quality evaluators, similar to the QAEvaluator. The key difference is that the Chat Evaluator is specifically designed for evaluating chat messages.

In [ ]:
from promptflow.evals.evaluators import ChatEvaluator

# Initialize Chat Evaluator
chat_eval = ChatEvaluator(model_config=model_config)

In [ ]:
conversation = [
    {"role": "user", "content": "What is the value of 2 + 2?"},
    {
        "role": "assistant",
        "content": "2 + 2 = 4",
        "context": {
            "citations": [{"id": "doc.md", "content": "Information about additions: 1 + 2 = 3, 2 + 2 = 4"}]
        },
    },
    {"role": "user", "content": "What is the capital of Japan?"},
    {
        "role": "assistant",
        "content": "The capital of Japan is Tokyo.",
        "context": {
            "citations": [
                {
                    "id": "doc.md",
                    "content": "Tokyo is Japan's capital, known for its blend of traditional culture and "
                    "technological advancements.",
                }
            ]
        },
    },
]

# Running Chat Evaluator on chat conversation
score = chat_eval(conversation=conversation)

In [ ]:
print(score)

## 2. Custom Evaluator

After gaining a basic understanding of how the built-in evaluators work, let's explore how to create your own evaluator. In this section, we will discuss creating both a code-based custom evaluator and a prompt-based custom evaluator.

### 2.1 Define a Code based Evaluator

A code-based evaluator can be as simple as a Python function. In the example below, we will show you a simple evaluator that calculates the length of an answer.

In [ ]:
with open("custom/answer_length.py") as fin:
    print(fin.read())

In [ ]:
from custom.answer_length import answer_length
print(answer_length("some answer"))

### 2.2 Define a Prompty based Evaluator

Prompty is a file with .prompty extension for developing prompt template. The prompty asset is a markdown file with a modified front matter. The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [ ]:
with open("custom/apology.prompty") as fin:
    print(fin.read())

In [ ]:
from promptflow.client import load_flow

# load apology evaluatorfrom prompty
apology_eval = load_flow(source="custom/apology.prompty", model={"configuration": model_config})

In [ ]:
result = apology_eval(
    question="What is the capital of France?", answer="Paris"
)

In [ ]:
print(result)

## 3. Using Evaluate API to evaluate with data

In previous sections, we walked you through how to use built-in evaluators to evaluate a single row and how to define your own custom evaluators. Now, we will show you how to use these evaluators with the powerful `evaluate` API to assess an entire dataset.

First, let's take a peek at what the data looks like.

In [ ]:
import pandas as pd

data_path = "data.jsonl"

df = pd.read_json(data_path, lines=True)
df

Now, we will invoke the `evaluate` API using three evaluators:
1. `qa`, which is a composite evaluator.
2. `answer_length`, which is a custom code-based evaluator.
3. `apology`, which is a prompt-based evaluator.

Additionally, we have a column mapping to map the `truth` column from the dataset to `ground_truth`, which is accepted by the evaluator.

In [ ]:
from promptflow.evals.evaluate import evaluate

result = evaluate(
    data="data.jsonl",
    evaluators={
        "qa": qa_eval,
        "answer_length": answer_length,
        "apology": apology_eval
    },
    # column mapping
    evaluator_config={
        "default": {
            "ground_truth": "${data.truth}"
        }
    }
)


Finally, let's check the results produced by the evaluate API.

In [ ]:
from IPython.display import display, JSON

display(JSON(result))